In [575]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import maup
import json
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.ops import orient

In [450]:
economic_columns = [
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
    '60K_100K', '100K_125K', '125K_150K', '150K_MORE'
]
race_columns = [
    'TOT_POP22', 'NHSP_POP22', 'HSP_POP22', 
    'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 
    'ASN_NHSP22', 'HPI_NHSP22', 'OTH_NHSP22'
]

## Functions

In [518]:
def aggregate_economic_data(block_gdf, precincts_gdf, variables, epsg):
    """
    Parameters:
    - block_gdf (GeoDataFrame): GeoDataFrame containing the economic data per census block
    - precincts_gdf (GeoDataFrame): GeoDataFrame representing precincts to which data will be aggregated
    - variables (list): List of column names to be aggregated.

    Returns:
    - GeoDataFrame: The updated precincts_gdf with aggregated economic data.
    """
    precincts_gdf = precincts_gdf.to_crs(epsg=epsg)
    block_gdf = block_gdf.to_crs(epsg=epsg)
    assignment = maup.assign(block_gdf, precincts_gdf)
    
    precincts_gdf[variables] = block_gdf[variables].groupby(assignment).sum()
    
    weighted_sum = (block_gdf['MEDN_INC22'] * block_gdf['TOT_HOUS22']).groupby(assignment).sum()
    total_households = block_gdf['TOT_HOUS22'].groupby(assignment).sum()
    
    precincts_gdf['MEDN_INC22'] = weighted_sum / total_households
    precincts_gdf = precincts_gdf.fillna(0)
    
    return precincts_gdf


In [519]:
def aggregate_racial_data(block_gdf, precincts_gdf, variables, epsg):
    """
    Parameters:
    - block_gdf (GeoDataFrame): GeoDataFrame containing the racial data per census block
    - precincts_gdf (GeoDataFrame): GeoDataFrame representing precincts to which data will be aggregated
    - variables (list): List of column names to be aggregated

    Returns:
    - GeoDataFrame: The updated precincts_gdf with aggregated economic data.
    """
    precincts_gdf = precincts_gdf.to_crs(epsg=epsg)
    block_gdf = block_gdf.to_crs(epsg=epsg)
    
    assignment = maup.assign(block_gdf, precincts_gdf)
    
    precincts_gdf[variables] = block_gdf[variables].groupby(assignment).sum()
    
    if 'TOT_POP22' in variables:
        variables.remove('TOT_POP22')
    precincts_gdf['TOT_POP22'] = precincts_gdf[['NHSP_POP22', 'HSP_POP22']].sum(axis=1)
    
    precincts_gdf = precincts_gdf.fillna(0)
    
    return precincts_gdf


In [520]:
def aggregate_region_data(block_gdf, precincts_gdf, variables, epsg):
    """
    Parameters:
    - block_gdf (GeoDataFrame): GeoDataFrame containing population data per census block
    - precincts_gdf (GeoDataFrame): GeoDataFrame representing precincts to which data will be aggregated
    - variables (list): List of column names to be aggregated

    Returns:
    - GeoDataFrame: The updated precincts_gdf with aggregated economic and region type data.
    """

    precincts_gdf = precincts_gdf.to_crs(epsg=epsg)
    block_gdf = block_gdf.to_crs(epsg=epsg)

    assignment = maup.assign(block_gdf, precincts_gdf)

    precincts_gdf[variables] = block_gdf[variables].groupby(assignment).sum()
    
    if 'TOT_POP22' in variables:
        variables.remove('TOT_POP22')
    precincts_gdf['TOT_POP22'] = precincts_gdf[['NHSP_POP22', 'HSP_POP22']].sum(axis=1)

    precincts_gdf = precincts_gdf.fillna(0)

    def get_region_type(row):
        if row['TOT_POP22'] >= 5000:
            return 'urban'
        elif 2500 <= row['TOT_POP22'] < 5000:
            return 'suburban'
        else:
            return 'rural'

    precincts_gdf['region_type'] = precincts_gdf.apply(get_region_type, axis=1)

    return precincts_gdf


# Aggregate Census Block data to precincts for South Carolina:

## Get block-level GeoDataFrames

In [521]:
sc_block_inc_gdf = gpd.read_file(
    'raw/census_block/income/sc_inc_2022_bg_shape_file/sc_inc_2022_bg.shp'
)
sc_block_inc_gdf = sc_block_inc_gdf.to_crs(epsg=4326)
sc_block_geometry_gdf = sc_block_inc_gdf[
    ['GEOID','STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry']
]

## Get precinct-level GeoDataFrames

In [522]:
sc_precincts_gdf = gpd.read_file(
    'states/south_carolina/geodata/south_carolina_precincts.geojson'
)


## Get Census Block categories csv

In [526]:
sc_econ_df = pd.read_csv(
    'processed_individual/sc_econ_block.csv'
)
sc_race_df = pd.read_csv(
    'processed_individual/sc_race_block.csv'
)

## Aggerating census block geometry with household income population data to precincts

In [527]:
sc_econ_df = sc_econ_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
sc_block_geometry_gdf['GEOID'] = sc_block_geometry_gdf['GEOID'].astype(str)
sc_econ_df['GEOID'] = sc_econ_df['GEOID'].astype(str)
sc_block_geometry_gdf = sc_block_geometry_gdf.merge(sc_econ_df, on='GEOID')
sc_economic_block_gdf = sc_block_geometry_gdf[['GEOID', 'geometry'] + economic_columns].copy()

In [528]:
variables = [
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
    '60K_100K', '100K_125K', '125K_150K', '150K_MORE'
]

In [529]:
sc_precincts_econ_gdf = aggregate_economic_data(sc_economic_block_gdf, sc_precincts_gdf, variables, epsg=2273)


### Drop geometry and export json

In [530]:
sc_precincts_econ_df = sc_precincts_econ_gdf.drop(columns=['geometry'])


In [531]:
with open('states/south_carolina/economic/south_carolina_precincts_household_income.json', 'w') as json_file:
    json.dump(sc_precincts_econ_df.to_dict(orient='records'), json_file, indent=4)

## Aggerating census block geometry with racial population data to precincts

In [532]:
sc_race_df = sc_race_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
sc_block_geometry_gdf['GEOID'] = sc_block_geometry_gdf['GEOID'].astype(str)
sc_race_df['GEOID'] = sc_race_df['GEOID'].astype(str)
sc_block_geometry_gdf = sc_block_geometry_gdf.merge(sc_race_df, on='GEOID')
sc_race_block_gdf = sc_block_geometry_gdf[['GEOID', 'geometry'] + race_columns].copy()

In [533]:
variables = [
    'TOT_POP22', 'NHSP_POP22', 'HSP_POP22', 'WHT_NHSP22',
    'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22', 'OTH_NHSP22'
]

In [534]:
sc_precincts_race_gdf = aggregate_racial_data(sc_race_block_gdf, sc_precincts_gdf, variables, epsg=2273)

### Drop geometry and export json

In [537]:
sc_precincts_race_df = sc_precincts_race_gdf.drop(columns=['geometry'])

In [538]:
with open('states/south_carolina/demographics/south_carolina_precincts_racial_population.json', 'w') as json_file:
    json.dump(sc_precincts_race_df.to_dict(orient='records'), json_file, indent=4)

## Aggerating block population data to precincts with region type

In [539]:
variables = ['TOT_POP22', 'NHSP_POP22', 'HSP_POP22']

In [541]:
sc_precincts_region_gdf = aggregate_region_data(sc_race_block_gdf, sc_precincts_gdf, variables, epsg=2273)

In [542]:
sc_precincts_region_df = sc_precincts_region_gdf[['UNIQUE_ID', 'region_type', 'TOT_POP22']]

In [543]:
print(sc_precincts_region_df['region_type'].value_counts())

region_type
rural       1413
suburban     689
urban        156
Name: count, dtype: int64


### Export json

In [544]:
with open('states/south_carolina/geodata/south_carolina_precincts_region_type.json', 'w') as json_file:
    json.dump(sc_precincts_region_df.to_dict(orient='records'), json_file, indent=4)

# Aggregate Census Block data to precincts for Maryland:

## Get block level geometry

In [554]:
md_block_inc_gdf = gpd.read_file(
    'raw/census_block/income/md_inc_2022_bg_shape_file/md_inc_2022_bg.shp'
)
md_block_geometry_gdf = md_block_inc_gdf[
    ['GEOID','STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry']
]

In [555]:
# check crs
print(md_block_geometry_gdf.crs)

EPSG:4269


## Get precincts level geoDataFrames

In [586]:
md_precincts_gdf = gpd.read_file('states/maryland/geodata/maryland_precincts.geojson')

## Get Census Block categories csv

In [557]:
md_econ_df = pd.read_csv('processed_individual/md_econ_block.csv')
md_race_df = pd.read_csv('processed_individual/md_race_block.csv')

## Aggerating census block geometry with household income population data to precincts

In [558]:
md_econ_df = md_econ_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
# md_block_geometry_gdf['GEOID'] = md_block_geometry_gdf['GEOID'].astype(str)
md_econ_df['GEOID'] = md_econ_df['GEOID'].astype(str)
md_block_geometry_gdf = md_block_geometry_gdf.merge(md_econ_df, on='GEOID')
md_economic_block_gdf = md_block_geometry_gdf[['GEOID', 'geometry'] + economic_columns].copy()

In [559]:
variables = [
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
    '60K_100K', '100K_125K', '125K_150K', '150K_MORE'
]

In [564]:
md_economic_block_gdf = md_economic_block_gdf.to_crs(epsg=2248)

In [565]:
maup.doctor(md_economic_block_gdf)

True

In [587]:
md_precincts_gdf = md_precincts_gdf.to_crs(epsg=2248)

In [589]:

md_precincts_gdf['geometry'] = md_precincts_gdf.make_valid()
md_precincts_gdf = md_precincts_gdf[~md_precincts_gdf.is_empty].dropna(subset=['geometry'])
md_precincts_gdf = md_precincts_gdf.explode(ignore_index=True)
maup.doctor(md_precincts_gdf)


There are 788 overlaps.
There are 6697 holes.


False

In [563]:
md_precincts_econ_gdf = aggregate_economic_data(md_economic_block_gdf, md_precincts_gdf, variables, epsg=2248)


GEOSException: TopologyException: side location conflict at 1369643.0950017923 323059.40850947 0. This can occur if the input geometry is invalid.

### Drop geometry and export json

In [474]:
md_precincts_econ_gdf = md_precincts_econ_gdf.drop(columns=['geometry'])


In [475]:
with open('states/maryland/economic/maryland_precincts_household_income.json', 'w') as json_file:
    json.dump(md_precincts_econ_gdf.to_dict(orient='records'), json_file, indent=4)

## Aggerating census block geometry with racial population data to precincts

In [ ]:
md_race_df = md_race_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
md_race_df['GEOID'] = md_race_df['GEOID'].astype(str)
md_block_geometry_gdf = md_block_geometry_gdf.merge(md_race_df, on='GEOID')
md_race_block_gdf = md_block_geometry_gdf[['GEOID', 'geometry'] + race_columns].copy()

Index(['GEOID', 'STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry',
       'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K', '60K-100K', '100K_125K',
       '125K_150K', '150K_MORE', 'TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22'],
      dtype='object')


In [408]:
variables = ['TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22']

In [491]:
md_race_precincts_gdf = aggregate_racial_data(md_race_block_gdf, md_precincts_gdf, variables)

NameError: name 'md_race_block_gdf' is not defined

### Drop geometry and export json

In [ ]:
md_race_precincts_gdf.drop(columns=['geometry'], inplace=True)

In [412]:
with open('states/maryland/demographics/maryland_precincts_racial_population.json', 'w') as json_file:
    json.dump(md_race_precincts_gdf.to_dict(orient='records'), json_file, indent=4)

## Aggerating block population data to precincts with region type

In [413]:
variables = ['TOT_POP22', 'NHSP_POP22', 'HSP_POP22']

In [ ]:
md_precincts_region_gdf = aggregate_region_data(md_race_block_gdf, md_precincts_gdf, variables)

/Users/tonymac/miniconda3/envs/cse354/lib/python3.11/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/Users/tonymac/miniconda3/envs/cse354/lib/python3.11/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/tonymac/miniconda3/envs/cse354/lib/python3.11/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).ar

In [417]:
print(md_precincts_region_gdf['region_type'].value_counts())

region_type
rural       913
suburban    786
urban       344
Name: count, dtype: int64


### Drop geometry and export json

In [419]:
# drop geometry
md_precincts_region_gdf.drop(columns=['geometry'], inplace=True)

In [ ]:
md_precincts_region_gdf = md_precincts_region_gdf[['UNIQUE_ID', 'region_type', 'TOT_POP22']]

In [421]:

with open('states/maryland/geodata/maryland_precincts_region_type.json', 'w') as json_file:
    json.dump(md_precincts_region_gdf.to_dict(orient='records'), json_file, indent=4)